In [1]:
import os
import shutil

directory_out = './file/5.collate'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [2]:
directory_in = './file/4.info'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    # = = = = = = = = = = = = = = =

    list_series, list_column, list_column_note, list_column_note_2, list_column_price, list_column_specification = [], df_input.columns.tolist(), ['Note 1'], ['Note 2'], ['Price'], []
    for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
        dict_note, dict_note_2, dict_price, dict_dict = json.loads(series['Json_Note 1']), json.loads(series['Json_Note 2']), json.loads(series['Json_Price']), json.loads(series['Json_Specification'])

        if len(dict_note) < 2:
            series['Note 1'] = dict_note.get('0', '')
        else:
            for key, value in dict_note.items():
                key = f'Note 1-{int(key) + 1}'

                if key not in list_column_note:
                    list_column_note.append(key)

                series[key] = value

        # = = = = = = = = = = = = = = =

        if len(dict_note_2) < 2:
            series['Note 2'] = dict_note_2.get('0', '')
        else:
            for key, value in dict_note_2.items():
                key = f'Note 2-{int(key) + 1}'

                if key not in list_column_note_2:
                    list_column_note_2.append(key)

                series[key] = value

        # = = = = = = = = = = = = = = =

        series['Price'] = dict_price.get('0', '')
        for key, value in dict_price.get('1', {}).items():
            key = f'Price-{int(key) + 2}'

            if key not in list_column_price:
                list_column_price.append(key)

            series[key] = value

        # = = = = = = = = = = = = = = =

        for dict_ in dict_dict.values():
            if dict_['0'] in list_column:
                dict_['0'] = f'''{dict_['0']} (Specification)'''

            if dict_['0'] not in list_column_specification:
                list_column_specification.append(dict_['0'])

            series[dict_['0']] = dict_.get('1', 'N/A')

        list_series.append(series)

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                      ascending=[True],
                                                      ignore_index=True)
    df_output[list_column[:8] + list_column_note + list_column_note_2 + list_column[12:13] + list_column_price + sorted(list_column_specification) + list_column[8:12] + list_column[13:]].to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 485.25it/s]


crawler_2.xlsx


Progress: 100%|████████████████████████████| 10/10 [00:00<00:00, 1075.74it/s]


Done ~


# 2. Vehicle

In [3]:
directory_in = './file/3.vehicle'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =

    df_input = pd.read_excel(f'{directory_in}/{file}',
                             header=0,
                             dtype=str).fillna('')

    df_input['Vehicle No'] = df_input['Vehicle No'].astype(int)
    df_input['Year'] = df_input['Year'].astype(int)

    # = = = = = = = = = = = = = = =

    list_dict, grouped = [], df_input.groupby('JOIN_MPNTCPC')
    for mpntcpc, group in tqdm(grouped, desc='Progress', ncols=77):
        df_temp = group.reset_index(drop=True)

        dict_vehicle = {}
        for i in range(len(df_temp)):
            make, model, year = df_temp.loc[i, 'Make'], df_temp.loc[i, 'Model'], df_temp.loc[i, 'Year']

            if make not in dict_vehicle:
                dict_vehicle[make] = {model: [year]}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            elif year not in dict_vehicle[make][model]:
                dict_vehicle[make][model].append(year)

        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make, dict_model in dict_vehicle.items():
            for model, list_year in dict_model.items():
                year_begin, year_end = min(list_year), max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        list_dict.append({'JOIN_MPNTCPC': mpntcpc,
                          'Vehicle': '\n'.join(sorted(list_vehicle))})

    # = = = = = = = = = = = = = = =

    df_output = pd.DataFrame(list_dict).sort_values(by=['JOIN_MPNTCPC'],
                                                    ascending=[True],
                                                    ignore_index=True)
    df_output.to_excel(f'{directory_out}/{file}', index=False)

print()
print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|████████████████████████████| 10/10 [00:00<00:00, 1078.87it/s]


crawler_2-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 956.58it/s]


Done ~
